In [55]:
import numpy as np
from numpy import empty
import random
import matplotlib.pyplot as plt
import scipy.constants
from scipy.integrate import odeint
from functools import partial
from IPython.core.debugger import set_trace

## Randomly generate individual set of parameters within specified values

In [56]:
def generate_individual():
    
    L = random.uniform(5, 15)
    P = random.uniform(20e-3, 200e-3)
#   W = random.choice([980, 1480])
    W = 980
    
    individual = [L, P, W]
    
    return individual

## Generate initial population

In [78]:
def generate_initial_population(size):
    
    pop = np.zeros([size, 3])
    
    for i in range(size):
        
        pop[i] = generate_individual()
        
    return pop

population = generate_initial_population(100)

len(population)

100

## Determine fitness levels of the generation

In [106]:
def fitness_calc(population):
    
    solution = np.zeros(len(population))
    
    for i in range(len(population)):
    
#       solution[i] = ((individual[i][0]*individual[i][1]*individual[i][2])/3)
        solution[i] = EDFA_Model(population[i][0], population[i][1], population[i][2])

    fitness_array = np.zeros(len(solution))
    
    for i in range(len(solution)):
    
        fitness_array[i] = solution[i] 

    return fitness_array

## Select new mating pool based on fittest genes

In [117]:
def select_mating_pool(population, fitness, num_parents):
    
    parents = np.zeros([num_parents, len(population[1])])
    
    for parent_num in range(num_parents):
        
        max_fitness_i = np.where(fitness == np.min(fitness))
        
        max_fitness_i = max_fitness_i[0][0]
        
        parents[parent_num, :] = population[max_fitness_i, :]
        
        fitness[max_fitness_i] = 999999999
        
    return parents

## Mix genes to produce offspring

In [146]:
def crossover(parents, offspring_size):
    
    offspring = np.zeros(len(offspring_size))
    print(range(len(offspring_size)))                   
    crossover_point = np.uint8(len(offspring_size[1])/2)
 
    for i in range(len(offspring_size)):
        
        parent1_i = i%parents.shape[0]
        parent2_i = (i+1)%parents.shape[0]
                         
        offspring[i, 0:crossover_point] = parents[parent1_i, 0:crossover_point]
                         
        offspring[i, crossover_point:] = parents[parent2_i, crossover_point:]
                         
    return offspring

## Mutate 1 gene (in this case fibre length)

In [63]:
def mutation(offspring_crossover):
    
    for i in range(offspring_crossover):
        
        random_value = np.random.uniform(-3, 3)

        offspring_crossover[i, 0] = offspring_crossover[i, 0] + random_value
    
    return offspring_crossover

## Wrapper function 

In [147]:
def ga_wrapper(num_generations, num_parents_mating, pop_size):
    
    population = generate_initial_population(pop_size) 
    
    num_generations = 5
    
    generations = np.zeros(num_generations)
    
    for generation in range(num_generations):
        
        fitness_array = fitness_calc(population)
        
        parents = select_mating_pool(population, fitness_array, num_parents_mating)

        offspring_crossover = crossover(parents, offspring_size=(pop_size-parents))
                                        
        offspring_mutation = mutation(offspring_crossover)                    
                                        
        generations[generation] = zip(population, fitness_array)
                                        
        population[0:len(parents), :] = parents
        population[len(parents):, :] = offspring_mutation
                                        
    return generations
                                        
ga_wrapper(5, 4, 200)
ga_wrapper

[[ 2.00000000e-02  6.87172553e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.18694904e-03  2.31818323e-03 -2.89638970e-75 -2.89638970e-75
  -2.89638970e-75  7.05739468e-78  7.05739468e-78  7.05739468e-78]
 [ 7.04426564e-05  7.82052694e-05 -8.24160024e-73 -8.24160024e-73
  -8.24160024e-73  4.03649855e-78  4.03649855e-78  4.03649855e-78]
 [ 4.18081272e-06  2.63834056e-06 -2.34498929e-70 -2.34498929e-70
  -2.34498929e-70  2.29578012e-78  2.29578012e-78  2.29578012e-78]
 [ 2.48553363e-07  8.92845717e-08 -6.78373938e-68 -6.78373938e-68
  -6.78373938e-68  1.33135759e-78  1.33135759e-78  1.33135759e-78]
 [ 1.49199494e-08  3.28442254e-09 -2.17714960e-65 -2.17714960e-65
  -2.17714960e-65  6.07312634e-79  6.07312634e-79  6.07312634e-79]
 [ 9.85504537e-10  1.52800575e-10 -6.50813352e-63 -6.50813352e-63
  -6.50813352e-63  4.34496128e-79  4.34496128e-79  4.34496128e-79]
 [-4.25329115e-10 -7.82727540e-11  3.55127128e-63  3.55127128e-63
   

[[ 2.00000000e-02  7.56997449e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.41069600e-03  1.93261229e-02 -1.00628021e-76 -1.00628021e-76
  -1.00628021e-76  9.54748417e-78  9.54748417e-78  9.54748417e-78]
 [ 2.05485016e-03  4.93394555e-03 -1.05958077e-75 -1.05958077e-75
  -1.05958077e-75  8.58532821e-78  8.58532821e-78  8.58532821e-78]
 [ 6.58650303e-04  1.25963155e-03 -1.03770793e-74 -1.03770793e-74
  -1.03770793e-74  6.93878344e-78  6.93878344e-78  6.93878344e-78]
 [ 2.11120090e-04  3.21582734e-04 -1.01054576e-73 -1.01054576e-73
  -1.01054576e-73  5.53693255e-78  5.53693255e-78  5.53693255e-78]
 [ 6.76711555e-05  8.20992742e-05 -9.83697893e-73 -9.83697893e-73
  -9.83697893e-73  4.41110545e-78  4.41110545e-78  4.41110545e-78]
 [ 2.16912969e-05  2.09606504e-05 -9.57677863e-72 -9.57677863e-72
  -9.57677863e-72  3.51388887e-78  3.51388887e-78  3.51388887e-78]
 [ 6.95292570e-06  5.35107892e-06 -9.32372006e-71 -9.32372006e-71
  -

[[ 2.00000000e-02  2.30276289e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.48570920e-03  1.01703477e-03 -6.18737089e-76 -6.18737089e-76
  -6.18737089e-76  2.46593820e-78  2.46593820e-78  2.46593820e-78]
 [ 1.10367092e-04  4.49186041e-05 -1.12520807e-73 -1.12520807e-73
  -1.12520807e-73  1.47976090e-78  1.47976090e-78  1.47976090e-78]
 [ 8.19713461e-06  1.98249397e-06 -2.04456303e-71 -2.04456303e-71
  -2.04456303e-71  8.79753515e-79  8.79753515e-79  8.79753515e-79]
 [ 6.09162650e-07  8.79725711e-08 -3.71127957e-69 -3.71127957e-69
  -3.71127957e-69  5.20446989e-79  5.20446989e-79  5.20446989e-79]
 [ 4.37273424e-08  3.37560732e-09 -7.16911724e-67 -7.16911724e-67
  -7.16911724e-67  3.60648093e-79  3.60648093e-79  3.60648093e-79]
 [ 3.55533503e-09  2.08815597e-10 -1.35448199e-64 -1.35448199e-64
  -1.35448199e-64  2.49546054e-79  2.49546054e-79  2.49546054e-79]
 [ 1.58004025e-10  5.91738885e-12 -7.49794023e-63 -7.49794023e-63
  -

   2.40214412e-65  9.91588853e-79  9.91588853e-79  9.91588853e-79]]
[[ 2.00000000e-02  4.11420854e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.62795302e-03  2.02778058e-03 -9.20001746e-76 -9.20001746e-76
  -9.20001746e-76  4.47952583e-78  4.47952583e-78  4.47952583e-78]
 [ 1.32511036e-04  9.99427267e-05 -1.39427267e-73 -1.39427267e-73
  -1.39427267e-73  2.74203776e-78  2.74203776e-78  2.74203776e-78]
 [ 1.07864729e-05  4.92594379e-06 -2.10567047e-71 -2.10567047e-71
  -2.10567047e-71  1.66117410e-78  1.66117410e-78  1.66117410e-78]
 [ 8.77930073e-07  2.43025181e-07 -3.18908281e-69 -3.18908281e-69
  -3.18908281e-69  9.99829740e-79  9.99829740e-79  9.99829740e-79]
 [ 7.04677036e-08  1.15602921e-08 -5.17066890e-67 -5.17066890e-67
  -5.17066890e-67  5.91551723e-79  5.91551723e-79  5.91551723e-79]
 [ 6.14235146e-09  7.32288756e-10 -8.33422740e-65 -8.33422740e-65
  -8.33422740e-65  3.14871379e-79  3.14871379e-79  3.14871379e-79]
 

[[ 2.00000000e-02  8.74171677e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.10747445e-03  1.30812911e-02 -2.98815923e-76 -2.98815923e-76
  -2.98815923e-76  1.09095870e-77  1.09095870e-77  1.09095870e-77]
 [ 8.43566550e-04  1.95750730e-03 -7.30251053e-75 -7.30251053e-75
  -7.30251053e-75  8.40919474e-78  8.40919474e-78  8.40919474e-78]
 [ 1.73246118e-04  2.92924433e-04 -1.73300235e-73 -1.73300235e-73
  -1.73300235e-73  6.14659137e-78  6.14659137e-78  6.14659137e-78]
 [ 3.55804981e-05  4.38348700e-05 -4.10990962e-72 -4.10990962e-72
  -4.10990962e-72  4.47961952e-78  4.47961952e-78  4.47961952e-78]
 [ 7.30746717e-06  6.55943225e-06 -9.74790813e-71 -9.74790813e-71
  -9.74790813e-71  3.26515254e-78  3.26515254e-78  3.26515254e-78]
 [ 1.50033900e-06  9.80543195e-07 -2.31769180e-69 -2.31769180e-69
  -2.31769180e-69  2.38002441e-78  2.38002441e-78  2.38002441e-78]
 [ 3.08713536e-07  1.47732104e-07 -5.55848150e-68 -5.55848150e-68
  -

...

   5.35873246e-66  3.28897432e-79  3.28897432e-79  3.28897432e-79]]
[[ 2.00000000e-02  1.72572413e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.37480130e-03  4.37618237e-02 -2.32245030e-76 -2.32245030e-76
  -2.32245030e-76  2.17733091e-77  2.17733091e-77  2.17733091e-77]
 [ 2.03190395e-03  1.10973438e-02 -2.47077439e-75 -2.47077439e-75
  -2.47077439e-75  1.95346110e-77  1.95346110e-77  1.95346110e-77]
 [ 6.47648789e-04  2.81411782e-03 -2.44672790e-74 -2.44672790e-74
  -2.44672790e-74  1.57661668e-77  1.57661668e-77  1.57661668e-77]
 [ 2.06431418e-04  7.13616927e-04 -2.40954616e-73 -2.40954616e-73
  -2.40954616e-73  1.25661391e-77  1.25661391e-77  1.25661391e-77]
 [ 6.57978893e-05  1.80962321e-04 -2.37186751e-72 -2.37186751e-72
  -2.37186751e-72  9.99972378e-78  9.99972378e-78  9.99972378e-78]
 [ 2.09726105e-05  4.58904020e-05 -2.33515394e-71 -2.33515394e-71
  -2.33515394e-71  7.95630958e-78  7.95630958e-78  7.95630958e-78]
 

  -8.09574708e-64  7.77170056e-79  7.77170056e-79  7.77170056e-79]]
[[ 2.00000000e-02  1.68073536e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.25648519e-03  1.22569442e-02 -1.94787401e-75 -1.94787401e-75
  -1.94787401e-75  1.93615876e-77  1.93615876e-77  1.93615876e-77]
 [ 2.54585982e-04  8.93848175e-04 -1.54287871e-73 -1.54287871e-73
  -1.54287871e-73  1.27611293e-77  1.27611293e-77  1.27611293e-77]
 [ 2.87236048e-05  6.51857963e-05 -1.21240080e-71 -1.21240080e-71
  -1.21240080e-71  8.25163532e-78  8.25163532e-78  8.25163532e-78]
 [ 3.24080110e-06  4.75343251e-06 -9.52915774e-70 -9.52915774e-70
  -9.52915774e-70  5.33620307e-78  5.33620307e-78  5.33620307e-78]
 [ 3.65682072e-07  3.47145874e-07 -7.53591914e-68 -7.53591914e-68
  -7.53591914e-68  3.43095038e-78  3.43095038e-78  3.43095038e-78]
 [ 4.07656421e-08  2.44124414e-08 -6.20727247e-66 -6.20727247e-66
  -6.20727247e-66  2.27411040e-78  2.27411040e-78  2.27411040e-78]
 

[[ 2.00000000e-02  8.88436979e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.48496165e-04  2.00341050e-03 -7.33560245e-75 -7.33560245e-75
  -7.33560245e-75  8.55608503e-78  8.55608503e-78  8.55608503e-78]
 [ 3.59975510e-05  4.51775443e-05 -4.08073400e-72 -4.08073400e-72
  -4.08073400e-72  4.56333857e-78  4.56333857e-78  4.56333857e-78]
 [ 1.52679599e-06  1.01767074e-06 -2.27457588e-69 -2.27457588e-69
  -2.27457588e-69  2.42731935e-78  2.42731935e-78  2.42731935e-78]
 [ 6.41512790e-08  2.22452257e-08 -1.33431959e-66 -1.33431959e-66
  -1.33431959e-66  1.35151302e-78  1.35151302e-78  1.35151302e-78]
 [ 2.91899354e-09  6.57096881e-10 -8.29600670e-64 -8.29600670e-64
  -8.29600670e-64  5.93568370e-79  5.93568370e-79  5.93568370e-79]
 [-2.45004152e-10 -6.78171101e-11 -5.71304537e-63 -5.71304537e-63
  -5.71304537e-63  5.67618917e-79  5.67618917e-79  5.67618917e-79]
 [-3.65080827e-10 -7.37265231e-11 -6.38119843e-63 -6.38119843e-63
  -

   1.54032628e-64  5.70253584e-79  5.70253584e-79  5.70253584e-79]]
[[ 2.00000000e-02  5.80065210e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.13191614e-03  3.95152656e-03 -7.53857288e-76 -7.53857288e-76
  -7.53857288e-76  6.61962707e-78  6.61962707e-78  6.61962707e-78]
 [ 2.27252805e-04  2.69185032e-04 -6.68308261e-74 -6.68308261e-74
  -6.68308261e-74  4.30556297e-78  4.30556297e-78  4.30556297e-78]
 [ 2.42247376e-05  1.83383927e-05 -5.88480148e-72 -5.88480148e-72
  -5.88480148e-72  2.75276398e-78  2.75276398e-78  2.75276398e-78]
 [ 2.58169781e-06  1.24817657e-06 -5.19645690e-70 -5.19645690e-70
  -5.19645690e-70  1.76064625e-78  1.76064625e-78  1.76064625e-78]
 [ 2.75956943e-07  8.58141036e-08 -4.63648408e-68 -4.63648408e-68
  -4.63648408e-68  1.12412230e-78  1.12412230e-78  1.12412230e-78]
 [ 2.91553970e-08  5.86818285e-09 -4.39181750e-66 -4.39181750e-66
  -4.39181750e-66  5.82861881e-79  5.82861881e-79  5.82861881e-79]
 

IndexError: too many indices for array

In [67]:
num_generations = 5

for generation in range(num_generations):
        print(generation)

0
1
2
3
4


In [ ]:
def print_results(generations)

    for i in range(len(generations)):
        
        print("Generation {}".format(i))
        print()
        print("Parameters:")
        print()
        print(generations[i].population)
        print("Fitness Values:")
        print()
        print(generations[i].fitness_array)
        print()
        print("Best result: {}".format(min(abs(generations[i].fitness_array))))

NameError: name 'ga_wrapper' is not defined

In [84]:
def EDFA_Model(fibre_length, pump_power, pump_wavelength):

    length = fibre_length
    Pp = pump_power
    wl = pump_wavelength

    def finding_N1_N2 (P_s, #power of signal
                   P_p, #power of pump
                   P_A_j, #array of powers of ASE noise at given frequencies
                   nu_s, #frequency of signal
                   nu_p, #frequency of pump
                   nu_j, #array of ASE frequencies 
                   sigma_s_a, sigma_s_e, #absorption and emission coeff for signal
                   sigma_p_a, sigma_p_e, #absorption and emission coeff for pump
                   sigma_nu_j_a,sigma_nu_j_e, #absorption and emission coeff arrays for ASE frequencies
                   overlap_s, #overlap factor for signal
                   overlap_p, #overlap factor for pump
                   overlap_nu_j, #array of overlap factors for ASE frequencies
                   N=0.7e-19,
                   tau=10.2e-3,
                   A=3e-6): 

        const = tau/(A*scipy.constants.h)
        ASE_comp = const*np.divide(np.multiply(overlap_nu_j,P_A_j),nu_j)
        sig_comp = const*overlap_s*P_s/nu_s
        pump_comp = const*overlap_p*P_p/nu_p
        numerator = sig_comp*sigma_s_a+np.sum(np.multiply(ASE_comp,sigma_nu_j_a))+pump_comp*sigma_p_a
        denomerator = (sig_comp*(sigma_s_a+sigma_s_e)+
                       np.sum(np.multiply(ASE_comp,(np.add(sigma_nu_j_a,sigma_nu_j_e))))
                       +pump_comp*(sigma_p_a+sigma_p_e))
        N2 = N*(numerator/denomerator)
        N1 = N-N2
        return N1,N2

    n_freq =3 

    kwargs_finding_N1_N2 ={
       'nu_s':1550, #wavelength of signal (nm)
       'nu_p':wl, #wavelength of pump(nm)
       'nu_j':list(range(1549, 1549+n_freq)), #array of ASE frequencies (nm)
       'sigma_s_a':2.91e-34, 'sigma_s_e':4.01e-18, #absorption and emission coeff for signal (m^2)
       'sigma_p_a':2.788e-25, 'sigma_p_e':0.08e-25, #absorption and emission coeff for pump (m^2)
       'sigma_nu_j_a':[2.91e-34]*n_freq,'sigma_nu_j_e':[4.01e-18]*n_freq,#absorption and emission coeff arrays for ASE frequencies
       #'alpha_nu_j':[2]*n_freq, #array of attenuations for ASE frequencies (dB/Km)
       'overlap_s':0.4, #overlap factor for signal
       'overlap_p':0.64, #overlap factor for pump
       'overlap_nu_j':[0.4]*n_freq, #array of overlap factors for ASE frequencies
       'N':0.7e-19,
       'tau':10.2e-3,
       'A':3e-6} # bandwidth ASE (nm)}



    N1,N2 = finding_N1_N2(1,1,[0,0,0],**kwargs_finding_N1_N2)

    Powers =[0.02,Pp]+[0]*n_freq*2

    def propagation_eq (Powers,
                       nu_s, #frequency of signal
                       nu_p, #frequency of pump
                       nu_j, #array of ASE frequencies 
                       sigma_s_a, sigma_s_e, #absorption and emission coeff for signal
                       sigma_p_a, sigma_p_e, #absorption and emission coeff for pump
                       sigma_nu_j_a,sigma_nu_j_e,#absorption and emission coeff arrays for ASE frequencies
                       alpha_nu_j, #array of attenuations for ASE frequencies (dB/Km)
                       overlap_s, #overlap factor for signal
                       overlap_p, #overlap factor for pump
                       overlap_nu_j, #array of overlap factors for ASE frequencies
                       N=0.7e-19,
                       tau=10.2e-3,
                       A=3e-6,
                       alpha_s=2, # attenuation of signal (dB/Km)
                       alpha_p=2.4,#attenuation of pump(dB/Km)
                       delta_nu_j=1): # bandwidth ASE (nm)

        P_s= Powers[0] #power of signal
        P_p= Powers[1] #power of pump
        P_A_j_minus= Powers[2:int((len(Powers)-2)/2)+2] #array of powers of ASE noise at
                                                        #given frequencies backward travelling

        P_A_j_plus = Powers[int((len(Powers)-2)/2)+2:]#array of powers of ASE noise at 
                                                      #given frequencies forward travelling

        P_A_j = np.add(P_A_j_minus, P_A_j_plus) #array of powers of ASE noise at given frequencies forward

        N1,N2 = finding_N1_N2 (P_s,P_p, P_A_j, nu_s, nu_p, nu_j, sigma_s_a, sigma_s_e, sigma_p_a, sigma_p_e, 
                               sigma_nu_j_a,sigma_nu_j_e, overlap_s, overlap_p, overlap_nu_j, 
                               N, tau, A) 

        #print(N1,N2)
        def finding_d_power(sigma_a, sigma_e, overlap, P, alpha):
            return (N2*sigma_e-N1*sigma_a)*overlap*P-alpha*P

        dP_p = finding_d_power(sigma_p_a,sigma_p_e, overlap_p,P_p, alpha_p)
        dP_s = finding_d_power(sigma_s_a,sigma_s_e, overlap_s,P_s, alpha_s) 
        #print(dP_s, dP_p)

        def find_d_powers_ASE(sigma_a, sigma_e, P, alpha,nu):    
            def d_power_ASE(sigma_a, sigma_e, P, alpha,nu):
                return (finding_d_power(sigma_a, sigma_e, overlap_s, P, alpha)
                        +N2*sigma_e*overlap_s*scipy.constants.h*delta_nu_j
                       -alpha*P)
            return np.array(list(map(lambda i: d_power_ASE(sigma_a[i], sigma_e[i], P[i], alpha[i],nu[i]),
                                     range(len(sigma_a)))))



        dP_A_j_plus = find_d_powers_ASE(sigma_nu_j_a,sigma_nu_j_e,P_A_j_plus,alpha_nu_j,nu_j)
        dP_A_j_minus = -find_d_powers_ASE(sigma_nu_j_a,sigma_nu_j_e,P_A_j_minus,alpha_nu_j,nu_j)
        #print(dP_s, dP_p, dP_A_j_plus, dP_A_j_minus)

        dPowers = np.hstack([[dP_s,dP_p],dP_A_j_minus,dP_A_j_plus])

        return dPowers


    n_freq =3 
    kwargs_propagation_eq ={
       'nu_s':1550, #frequency of signal (nm)
       'nu_p':980, #frequency of pump(nm)
       'nu_j':list(range(1549, 1549+n_freq)), #array of ASE frequencies (nm)
       'sigma_s_a':2.91e-34, 'sigma_s_e':4.01e-18, #absorption and emission coeff for signal (m^2)
       'sigma_p_a':2.788e-25, 'sigma_p_e':0.08e-25, #absorption and emission coeff for pump (m^2)
       'sigma_nu_j_a':[2.91e-34]*n_freq,'sigma_nu_j_e':[4.01e-18]*n_freq,#absorption and emission coeff arrays for ASE frequencies
       'alpha_nu_j':[2]*n_freq, #array of attenuations for ASE frequencies (dB/Km)
       'overlap_s':0.4, #overlap factor for signal
       'overlap_p':0.64, #overlap factor for pump
       'overlap_nu_j':[0.4]*n_freq, #array of overlap factors for ASE frequencies
       'N':0.7e-19,
       'tau':10.2e-3,
       'A':3e-6,
       'alpha_s':2, # attenuation of signal (dB/Km)
       'alpha_p':2.4,#attenuation of pump(dB/Km)
       'delta_nu_j':1} # bandwidth ASE (nm)}


    def solve_field_eq(P_0,z,kwargs):
        eq_to_solve = lambda P,z:propagation_eq(P,**kwargs)
        powers= odeint(eq_to_solve,P_0,z)
        print(powers)
        return powers,z


    p,z =solve_field_eq(Powers,np.linspace(0,length,10),kwargs_propagation_eq);


    def power_ASE(Powers):

        P_ASE = np.sum(Powers[:,+2:], axis=1) #array of powers of ASE noise at given frequencies forward
        return P_ASE

    def gain(Powers):
        return Powers[:,0]/Powers[0,0]

    def pow2db(value):
        return 10*np.log10(value)

    def db2pow(value):
        return np.power(10,value/10)

    def SNR(Powers):
        p_ase = power_ASE(Powers)
        p_ase[0] = -1e-75
        return np.divide(Powers[:,0],p_ase)


    def NF (Powers):
        snr = SNR(Powers)
    #     try:
        return snr[-1]/snr[0] #SNRout is the last element of Powers and SNRin is the first element of Powers
    #     except:
    #         return none
    # NoiseFigure = NF(p)
    # NoiseFigure

    SNRa = SNR(p)
    SNRa

    NoiseFigure = NF(p)
    NoiseFigure

    return NoiseFigure



# Ignore everything below

In [78]:
solution = np.zeros([100, 1])

for i in range(len(population)):
    solution[i] = ((population[i][0]*population[i][1]*population[i][2])/3)
    
solution

array([[1332.46626224],
       [ 973.63287382],
       [  79.36291052],
       [ 390.99796817],
       [1140.55085629],
       [ 320.68851849],
       [ 306.44094028],
       [ 638.76865307],
       [ 151.69384099],
       [ 290.57775422],
       [ 132.444243  ],
       [ 330.28261131],
       [ 438.99325034],
       [ 446.7887986 ],
       [ 675.2130767 ],
       [ 484.50327072],
       [ 384.36988243],
       [ 422.58849295],
       [ 787.86931186],
       [ 596.76710286],
       [ 120.0994219 ],
       [ 132.14826313],
       [1229.70210073],
       [ 748.67401561],
       [ 331.21427149],
       [ 247.36161305],
       [ 418.05680648],
       [ 811.18462651],
       [ 336.94947546],
       [ 669.42895448],
       [ 170.63452128],
       [ 944.46167731],
       [ 355.46424687],
       [ 483.61744084],
       [1125.86333891],
       [ 101.64283309],
       [ 271.87839655],
       [ 428.54073696],
       [1023.34843653],
       [ 717.92611794],
       [  98.41624957],
       [ 833.392

In [73]:
np.mean(solution)

485.1409107132838

In [82]:
a = fitness(population)
min(abs(a))

array([0.63764])

In [89]:
parents = np.empty((10, 3))
parents

array([[1.05475334e-311, 9.58487353e-322, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 5.02034658e+175],
       [3.79823469e+175, 1.16391660e+165, 1.79584374e-052],
       [4.08594369e+179, 5.01163111e+217, 5.49419094e-143],
       [9.80058441e+252, 1.30304358e-142, 2.14027814e+161],
       [5.04621383e+180, 4.50618615e-144, 7.79952704e-143],
       [6.01352666e-067, 2.62967465e+179, 8.70264083e-071],
       [3.79100468e+179, 1.41529403e+161, 9.16651763e-072],
       [4.31217112e+097, 1.14482968e-071, 3.08395576e+126],
       [9.30350598e+199, 3.67152475e-062, 1.58268989e-319]])

In [51]:
for i in range(5):
    print(i)

0
1
2
3
4


In [104]:
parents = (1, 10, 1)

# parent_num = 10
# parents[parent_num, :]

for i in range(len(parents)):
    print(i%parents)

TypeError: unsupported operand type(s) for %: 'int' and 'tuple'

In [103]:
offspring_size = 4

offspring = np.zeros(offspring_size)

offspring

array([0., 0., 0., 0.])

In [113]:
generations = np.arange(2, 11, 1)
generations

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [117]:
for i in range(len(generations)):
    print("Generation {}".format(i+1))
    print()
    print('i')

Generation 1

i
Generation 2

i
Generation 3

i
Generation 4

i
Generation 5

i
Generation 6

i
Generation 7

i
Generation 8

i
Generation 9

i


In [2]:
Powers =[0.02,0.3]+[0]*3*2
Powers

[0.02, 0.3, 0, 0, 0, 0, 0, 0]

[[ 2.00000000e-02  3.00000000e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.16736034e-03  2.08450283e-02 -3.77128202e-75 -3.77128202e-75
  -3.77128202e-75  3.43298848e-77  3.43298848e-77  3.43298848e-77]
 [ 2.34872306e-04  1.44837989e-03 -3.23564988e-73 -3.23564988e-73
  -3.23564988e-73  2.24146887e-77  2.24146887e-77  2.24146887e-77]
 [ 2.54525986e-05  1.00638029e-04 -2.75553197e-71 -2.75553197e-71
  -2.75553197e-71  1.43765009e-77  1.43765009e-77  1.43765009e-77]
 [ 2.75813169e-06  6.99161950e-06 -2.35026851e-69 -2.35026851e-69
  -2.35026851e-69  9.22097374e-78  9.22097374e-78  9.22097374e-78]
 [ 2.98901344e-07  4.86785217e-07 -2.00620062e-67 -2.00620062e-67
  -2.00620062e-67  5.87199171e-78  5.87199171e-78  5.87199171e-78]
 [ 3.23024180e-08  3.33990621e-08 -1.75733582e-65 -1.75733582e-65
  -1.75733582e-65  4.00255681e-78  4.00255681e-78  4.00255681e-78]
 [ 3.51357987e-09  2.45466122e-09 -1.61215373e-63 -1.61215373e-63
  -

1.2941220128655265e-19

In [36]:
p[:,0] 

array([ 2.00000000e-02,  2.16736034e-03,  2.34872306e-04,  2.54525986e-05,
        2.75813169e-06,  2.98901344e-07,  3.23024180e-08,  3.51357987e-09,
       -5.45450504e-11, -4.99226249e-10])